## Cohort Analysis

In [ ]:
import numpy as np 
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

#allows ploting of charts within the notebook
%matplotlib inline

# Sets all rows to display
pd.options.display.max_rows = None

In [ ]:
# Online Retail Dataset: https://archive.ics.uci.edu/ml/datasets/online+retail

In [ ]:
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
df.head(5)

In [ ]:
df.shape

### Cleaning Data

In [ ]:
df.isna().sum() #Checking out missing value

In [ ]:
df1 = df.dropna(subset=['CustomerID']) # dropping out missing values in column Customer ID
df1.shape

In [ ]:
df1.info()

## Munging and Wrangling data

In [ ]:
#Uses the datetime function to gets the month a datetime stamp and strips the time
def get_month(x):
    return dt.datetime(x.year, x.month, 1) #year, month, increments of day

In [ ]:
#Create a new column 
df1['InvoiceMonth'] = df1['InvoiceDate'].apply(get_month) 

In [ ]:
#Always inspect the data you've just created
df1['InvoiceMonth']

In [ ]:
#Create a CohortMonth column by grouping data and selecting the earliest instance in the data. 
df1['CohortMonth'] = df1.groupby('CustomerID')['InvoiceMonth'].transform('min')

In [ ]:
df1['CohortMonth']

In [ ]:
df1.head(5)

## Create the Cohort

In [ ]:
#When passed a datetime column this functions splits out year, month, day

def get_date(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [ ]:
#splits invoiced month and data into single variables
invoice_year, invoice_month, _ = get_date(df1, 'InvoiceMonth') 

In [ ]:
#Inspect the variable
invoice_month[:30] #[:30] selects the first 30 rows of data

In [ ]:
#Inspect the variable
invoice_year[:30] #[:30] selects the first 30 rows of data

In [ ]:
#splits cohort month and data into single variables
cohort_year, cohort_month, _ = get_date(df1, 'CohortMonth')

In [ ]:
cohort_month[:30]

In [ ]:
cohort_year[:30]

In [ ]:
# Creating a variable which holds the differnce between the invoice and cohort year 
year_diff = invoice_year - cohort_year

In [ ]:
year_diff

In [ ]:
# Creating a variable which holds the differnce between the invoice and cohort month 
month_diff = invoice_month - cohort_month

In [ ]:
month_diff

In [ ]:
#Now creating a column that has the calclation shows the 
df1['CohortIndex'] = year_diff * 12 + month_diff + 1

In [ ]:
df1['CohortIndex']

In [ ]:
df1.head(5)

## Cohort Analysis: Retention Rate

In [ ]:
#Group the data by columns CohortMonth','CohortIndex' then aggreate by column 'CustomerID'
cohort_data = df1.groupby(
    ['CohortMonth', 'CohortIndex'])['CustomerID'].apply(pd.Series.nunique).reset_index()

In [ ]:
#Take the cohort_data and plumb it into a Pivot Table. Setting index, columns and values as below.
cohort_count = cohort_data.pivot_table(index = 'CohortMonth',
                                       columns = 'CohortIndex',
                                       values = 'CustomerID')

In [ ]:
cohort_count

## Create retention as a percentage

In [ ]:
cohort_size = cohort_count.iloc[:,0] #select all the rows : select the first column
retention = cohort_count.divide(cohort_size, axis=0) #Divide the cohort by the first column
retention.round(3) # round the retention to 3 places

### Creating cohort heatmaps

In [ ]:
plt.figure(figsize = (11,9))
plt.title('Cohort Analysis - Retention Rate')
sns.heatmap(data = retention, 
            annot = True, 
            fmt = '.0%', 
            vmin = 0.0,
            vmax = 0.5,
            cmap = "YlGnBu")
plt.show()

### From the graph, it can be concluded that on 2011/11 some strategies might be implemented to increase the return users. About 50% of user from 2010/12 cohort return on 2011/11. Marketing analysts or product analysts might further investigate if some campaigns or feature changes that lead to the higher retention.

## Cohort Analysis: Average Quantity Sold

In [ ]:
cohort_data2 = df1.groupby(['CohortMonth', 'CohortIndex'])['Quantity'].mean().reset_index()
average_quantity  = cohort_data2.pivot_table(index = 'CohortMonth',
                                            columns = 'CohortIndex',
                                       values = 'Quantity').round(1)
average_quantity

In [ ]:
plt.figure(figsize = (11,9))
plt.title('Cohort Analysis - Average Quantity')
sns.heatmap(data = average_quantity, 
            annot = True, 
            cmap = "BuGn")
plt.show()

### It is observed that within the recent cohort/month, the average quantity on the website was facing a decline. It is a warning signal that users stop buying merchandise on the website. The reason needs further investigation.

## Cohort Analysis: Average sales

In [ ]:
#Create a new column for Total Sales 
df1['TotalSale'] = df1['Quantity'] * df1['UnitPrice']

In [ ]:
df1['TotalSale'].head()

In [ ]:
cohort_data3 = df1.groupby(['CohortMonth', 'CohortIndex'])['TotalSale'].mean().reset_index()
average_sales  = cohort_data3.pivot_table(index = 'CohortMonth',
                                           columns = 'CohortIndex',
                                      values = 'TotalSale').round(1)
average_sales

In [ ]:
plt.figure(figsize = (11,9))
plt.title('Cohort Analysis - Average Sales')
sns.heatmap(data = average_sales, 
            annot = True, 
            cmap = "Blues")
plt.show()

### The retailer faces a significant decline sales in recent months. Sales Team need to investigate the reason.